# Goals

### Use this to Evaluate TIF and Tax Abatements

#### STEP 1
 - Get all landing page info.
 - Most importantly:
  - Address
  - Name
  - Business Type?
  - Timeframe it operated
 - Also somewhat important. Get info into SQL. Query when it bombs & start where we left off

### Original static script - landing page only

This is the script that pulls from the indivdual page alone. Gets business info and officers

In [1]:
import requests, os, time, sqlite3
from bs4 import BeautifulSoup as bs
from IPython.display import display, clear_output

import pandas as pd

os.chdir('C:\\Users\\stob1\\Desktop\\Incremental Detroit\\Data projects\\Lara_Scrape\\')

db = 'LaraScrape.sqlite'


In [2]:
def pageCompare(ID):
    # Goal here is to plug this function into a loop for the range described above to copy records of all businesses 
    # available through this portal.
    
    baseLink = 'https://cofs.lara.state.mi.us/CorpWeb/CorpSearch/CorpSummary.aspx?ID='
    
    response = requests.get(baseLink + str(ID))
    soup = bs(response.text, 'lxml')
    message = soup.find(id = "MainContent_lblMessage")
    
    if message is None:
        return soup

def page_parse(page):
    # create empty dict with needed names and keys to pull from page(columns)
    catch = dict.fromkeys(Column_Names, [])
    
    for ID, name in zip(Column_IDs, Column_Names):
        catch[name]= page.find(id=ID)
        try:
            catch[name] = catch[name].get_text()
        except:
            pass
    return catch

def Officer_parse(page):
    # builds a separate table for officers
    
    # Grabs the html table - if it's there
    tble = page.find(id = 'MainContent_grdOfficers')#'MainContent_tblOfficers') # MainContent_grdOfficers grid #
    
    rows = []
    for x in tble.find_all('tr'):
        cell =[cell.get_text() for cell in x.find_all('td')]
        rows.append(cell)
    
    # purge any empty rows
    rows = [x for x in rows if len(x) > 0]
    
    # put records into DF
    df = pd.DataFrame.from_records(rows, columns = ['Title', 'Name', 'Address'])
    
    # Add Corp ID to every row in this DF
    ID = page.find(id='MainContent_lblIDNumberHeader').get_text()
    df['CorpID'] = ID
    
    with open('Lara_officers_Ordered.csv', 'a') as f:
            df.to_sql('LaraOfficers', conn, if_exists = 'append', index = False) 

def get_last_ID(database, table):
    # Grabs the last ID we left off with in a Database
    try:
        conn = sqlite3.connect(database)
        c = conn.cursor()
        c.execute('''SELECT ID_Number FROM '%s' ORDER BY ID_Number DESC LIMIT 1'''%(table))
        last_ID = c.fetchall()
        conn.close()
        return int(last_ID[0][0])
    except:
        return 800000000                       

In [3]:
Column_IDs = [
    'MainContent_lblEntityName','MainContent_lblIDNumberHeader','MainContent_lblEntityType','MainContent_lblOrganisationDate','MainContent_lblTerm',
    'MainContent_lblMostRecentAnnualReportYear','MainContent_lblInactiveDate','MainContent_tdRevivalDateText',
    'MainContent_lblMostRecentAnnualReportWithOfficersAndDirectors','MainContent_lblResidentAgentName','MainContent_lblResidentStreet',
    'MainContent_lblaptsuiteother','MainContent_lblResidentCity','MainContent_lblResidentState','MainContent_lblResidentZip','MainContent_lblPrincipleStreet',
    'MainContent_lblaptsuiteotherlblpricipal','MainContent_lblPrincipleCity','MainContent_lblPrincipleState','MainContent_lblPrincipleZip',
    'MainContent_lblActsFormedUnder','MainContent_lblSum','MainContent_txtComments']

Column_Names = [
    'EntityName','ID_Number','EntityType','OrganisationDate','Term','MostRecentAnnualReportYear','InactiveDate','RevivalDateText',
    'MostRecentAnnualReportWithOfficersAndDirectors','ResidentAgentName','ResidentStreet','aptsuiteother','ResidentCity','ResidentState','ResidentZip',
    'PrincipleStreet','aptsuiteotherlblpricipal','PrincipleCity','PrincipleState','PrincipleZip','ActsFormedUnder','Total_Auth_Shares','Comments']

In [ ]:
# Start process
page_ID = get_last_ID(db,'LaraBusiness')

#page_ID = 123456

print(page_ID)

while page_ID <= 803000000:
    # Set the begin at the ID we want to start at then rename that ID after every connection.
    # Retry at failure    - Think Ill need to add a for loop for that.
    #page_ID = get_last_ID(db,'LaraBusiness')
    clear_output(wait=True)
    display('Working on: '+str(page_ID))
    try:
        conn = sqlite3.connect(db)
        page_content = pageCompare(page_ID)

        if page_content is not None:

            page_information = page_parse(page_content)

            if page_content.find(id = 'MainContent_grdOfficers') is not None:
                Officer_parse(page_content)

            df = pd.DataFrame(page_information, index = [page_information['ID_Number']],columns = Column_Names)

            df.to_sql('LaraBusiness', conn, if_exists = 'append', index = False)

            # Sleep because it is commonly considered polite.
            time.sleep(2) 
        page_ID += 1
        conn.close()
    except:
        print('failure at -------- ' + str(page_ID))
        time.sleep(30)
        pass

'Working on: 800083976'